In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os.path, sys, re
import time
from PIL import Image

from sktime.utils.data_processing import (
    from_3d_numpy_to_nested,
    from_multi_index_to_3d_numpy,
    from_nested_to_3d_numpy,
    from_multi_index_to_nested,
    from_nested_to_multi_index,
)


In [2]:
cd ~/Desktop/Data_Science/Erdos_Institute/ecg-proj/ecg-copy/

/home/moise/Desktop/Data_Science/Erdos_Institute/ecg-proj/ecg-copy


In [3]:
pathroot = "CSV_data_v2/"

In [4]:
LeadDict={'Lead1':np.array([[]]),'Lead2':np.array([[]]),'Lead3':np.array([[]]),'Lead4':np.array([[]]),
          'Lead5':np.array([[]]),'Lead6':np.array([[]]),'Lead7':np.array([[]]),'Lead8':np.array([[]]),
         'Lead9':np.array([[]]),'Lead10':np.array([[]]),'Lead11':np.array([[]]),'Lead12':np.array([[]])}

In [5]:
leadMinLen = {'Lead1':0,'Lead2':0,'Lead3':0,'Lead4':0,'Lead5':0,'Lead6':0,
           'Lead7':0,'Lead8':0,'Lead9':0,'Lead10':0,'Lead11':0,'Lead12':0}

In [6]:
ClassLabels={'ECGImagesofPatientthathaveHistoryofMI':0,'ECGImagesofPatientthathaveabnormalheartbeat':1,
            'ECGImagesofCOVID-19Patients':2,'NormalPersonECGImages':3,'ECGImagesofMyocardialInfarctionPatients':4}

In [7]:
"""
For every "x" value of the signal, average the "y" values over duplicates.

Inputs:
-------
df  data frame of two columns containing the signal "x" and "y" coordinates corresponding to "active" pixels

Outputs:
--------
signal numpy array of unique values ("y") of the signal
--------

"""
def uniqValsSignal(df):
    xdf=df[0].to_numpy()
    ydf=df[1].to_numpy()
    unikVals = pd.unique(xdf)
    signal = np.zeros(len(unikVals))
    for i in range(len(unikVals)):
        mask = (xdf==unikVals[i])
        signal[i] = np.mean(ydf[mask])
    return signal

In [8]:
"""
Note:
-----

1) For the "time series", only the second column of every lead is extracted as signal.
2) Because of non-uniformity in signal lenght across both observations and Leads, the code
uses the minimum signal length across observation and leads, in order to make the date "proper
for multivariate time series classification.
"""

labelArr = np.array([])
for dirs in os.listdir(pathroot):
#     if dirs == 'ECGImagesofCOVID-19Patients':
#         continue
    t = time.time()
    count = 0
    print('Processing {0} folder ...'.format(dirs))
    if not os.path.isfile(dirs):
        for item in os.listdir(os.path.join(pathroot,dirs)):
            #print('Processing {0} file ...'.format(item))
            of, oe = os.path.splitext(item)
            if of[0]=='.':
                continue
            else:
                signal = pd.read_csv(os.path.join(pathroot,os.path.join(dirs,item)),header=None,sep=' ')
                signal = uniqValsSignal(signal)
            try:
                leadNum = int(of[-2:])
            except ValueError:
                leadNum = int(of[-1:])
            finally:
                leadKey = 'Lead'+str(leadNum)
            if leadNum == 13:
                continue
            if LeadDict[leadKey].shape[1] > 0:
                if len(signal)> leadMinLen[leadKey]:
                    signal = np.reshape(signal[:leadMinLen[leadKey]],(1,leadMinLen[leadKey]))
                    LeadDict[leadKey] = np.concatenate( (LeadDict[leadKey],signal) )
                else:
                    LeadDict[leadKey] = LeadDict[leadKey][:,:len(signal)]
                    LeadDict[leadKey] = np.concatenate( (LeadDict[leadKey],np.reshape(signal,(1,len(signal)))) )
                    leadMinLen[leadKey] = len(signal) 
            else:
                LeadDict[leadKey] = np.reshape(signal,(1,len(signal)))
                leadMinLen[leadKey] = len(signal)         
            count = count+1
        labelArr = np.append(labelArr,np.repeat(ClassLabels[dirs],len(LeadDict[leadKey])-len(labelArr))) ##Add labels
        t = time.time()-t
        print('{0} files processed in this folder in {1} sec...\n'.format(count,round(t)))    

Processing ECGImagesofPatientthathaveHistoryofMI folder ...
2064 files processed in this folder in 21 sec...

Processing ECGImagesofPatientthathaveabnormalheartbeat folder ...
2796 files processed in this folder in 32 sec...

Processing ECGImagesofCOVID-19Patients folder ...
3000 files processed in this folder in 16 sec...

Processing NormalPersonECGImages folder ...
3408 files processed in this folder in 44 sec...

Processing ECGImagesofMyocardialInfarctionPatients folder ...
2868 files processed in this folder in 48 sec...



In [9]:
"""
Post Processing:
---------------

1) Put all the leads on the same "time" scale/Length
2) Pull all the leads(2D) into a 3D array of shape (n_obs,n_col,n_timepoints)
3) Convert result from step 2 into a nested data frame.
"""

minLen = min(leadMinLen.values())
for key in LeadDict.keys():
    LeadDict[key] = LeadDict[key][:,:minLen]

dim1 = LeadDict['Lead1'].shape[0]
dim2 = len(LeadDict.keys())
dim3 = LeadDict['Lead1'].shape[1]
X3d = np.zeros((dim1,dim2,dim3))
for j in range(dim2):
    X3d[:,j,:] = list(LeadDict.values())[j]
    
X3d_nested=from_3d_numpy_to_nested(X3d)
X3d_nested.columns = list(LeadDict.keys())
X3d_nested['Label'] = labelArr
X3d_nested.to_csv('muti-lead-dataFrame.csv',index=False,float_format='%d')

In [10]:
X3d_nested

,Lead1,Lead2,Lead3,Lead4,Lead5,Lead6,Lead7,Lead8,Lead9,Lead10,Lead11,Lead12,Label
0,0 126.0 1 119.5 2 134.0 3 ...,0 105.0 1 104.5 2 104.5 3 ...,0 104.714286 1 117.250000 2 101...,0 35.957447 1 35.500000 2 46...,0 158.000000 1 157.333333 2 149...,0 119.5 1 116.5 2 125.5 3 ...,0 137.5 1 137.0 2 136.5 3 ...,0 156.0 1 154.0 2 153.5 3 ...,0 162.000000 1 162.000000 2 151...,0 155.0 1 154.5 2 155.5 3 ...,0 193.545455 1 3.500000 2 3...,0 89.000000 1 109.500000 2 91...,0.0
1,0 139.0 1 129.0 2 127.0 3 ...,0 103.0 1 103.0 2 102.0 3 ...,0 80.5 1 80.0 2 79.0 3 79....,0 67.5 1 72.0 2 77.0 3 ...,0 127.5 1 126.0 2 131.5 3 ...,0 130.0 1 139.0 2 130.5 3 ...,0 129.5 1 129.5 2 129.5 3 ...,0 155.5 1 155.5 2 155.5 3 ...,0 139.0 1 131.5 2 128.5 3 ...,0 153.5 1 149.0 2 151.0 3 ...,0 80.166667 1 77.071429 2 82...,0 109.000000 1 100.000000 2 111...,0.0
2,0 158.333333 1 158.333333 2 149...,0 103.5 1 106.5 2 110.5 3 ...,0 103.5 1 103.5 2 103.5 3 ...,0 97.5 1 97.5 2 97.0 3 ...,0 118.363636 1 97.000000 2 94...,0 116.666667 1 155.500000 2 156...,0 131.200000 1 136.166667 2 104...,0 108.5 1 100.0 2 112.5 3 ...,0 169.500000 1 153.000000 2 145...,0 156.0 1 156.0 2 157.5 3 ...,0 145.5 1 145.5 2 145.5 3 ...,0 107.170732 1 104.000000 2 121...,0.0
3,0 116.5 1 117.5 2 126.0 3 ...,0 100.5 1 100.5 2 101.5 3 ...,0 104.5 1 104.5 2 104.0 3 ...,0 63.0 1 60.0 2 67.5 3 ...,0 130.0 1 123.5 2 129.5 3 ...,0 131.5 1 130.0 2 131.5 3 ...,0 140.5 1 150.0 2 144.5 3 ...,0 155.0 1 155.0 2 155.0 3 ...,0 169.500000 1 153.000000 2 145...,0 119.5 1 122.0 2 129.5 3 ...,0 154.055944 1 136.396396 2 44...,0 121.714286 1 144.000000 2 133...,0.0
4,0 157.000000 1 157.000000 2 148...,0 106.5 1 106.5 2 106.0 3 ...,0 107.0 1 106.0 2 106.0 3 ...,0 58.5 1 70.0 2 86.0 3 ...,0 141.0 1 136.5 2 138.0 3 ...,0 155.5 1 158.5 2 156.5 3 ...,0 131.0 1 130.5 2 130.0 3 ...,0 133.5 1 133.0 2 146.5 3 ...,0 139.333333 1 159.357143 2 156...,0 137.0 1 138.5 2 145.5 3 ...,0 137.5 1 138.0 2 146.5 3 ...,0 140.0 1 135.0 2 142.0 3 ...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173,0 130.5 1 127.5 2 127.0 3 ...,0 100.5 1 100.0 2 99.5 3 ...,0 107.0 1 107.0 2 109.0 3 ...,0 57.0 1 67.5 2 82.5 3 ...,0 110.5 1 107.5 2 123.5 3 ...,0 142.5 1 145.0 2 156.5 3 ...,0 97.0 1 124.5 2 105.5 3 ...,0 127.5 1 126.0 2 127.5 3 ...,0 142.5 1 137.5 2 137.5 3 ...,0 133.0 1 132.5 2 132.5 3 ...,0 4.0 1 12.5 2 19.0 3 ...,0 138.5 1 143.0 2 143.5 3 ...,4.0
1174,0 92.5 1 78.5 2 92.0 3 ...,0 102.5 1 103.0 2 103.0 3 ...,0 87.5 1 87.5 2 87.5 3 ...,0 74.0 1 72.5 2 74.5 3 ...,0 71.5 1 74.5 2 83.0 3 ...,0 113.0 1 111.0 2 119.0 3 ...,0 141.600000 1 133.250000 2 122...,0 121.0 1 120.5 2 120.0 3 ...,0 87.0 1 86.5 2 86.0 3 ...,0 140.0 1 148.5 2 142.0 3 ...,0 137.909091 1 140.000000 2 137...,0 117.5 1 125.0 2 127.0 3 ...,4.0
1175,0 130.0 1 131.5 2 138.0 3 ...,0 103.0 1 103.5 2 103.5 3 ...,0 97.0 1 96.5 2 96.0 3 ...,0 54.5 1 40.0 2 70.0 3 ...,0 142.000 1 147.000 2 142.000 3...,0 135.0 1 145.0 2 138.5 3 ...,0 97.0 1 124.5 2 105.5 3 ...,0 137.5 1 144.5 2 153.5 3 ...,0 87.0 1 86.5 2 86.0 3 ...,0 156.5 1 156.5 2 155.5 3 ...,0 135.0 1 135.0 2 135.5 3 ...,0 128.0 1 124.5 2 136.5 3 ...,4.0
1176,0 127.0 1 133.5 2 137.5 3 ...,0 100.5 1 100.0 2 102.5 3 ...,0 91.5 1 94.5 2 96.0 3 ...,0 74.0 1 72.5 2 74.5 3 ...,0 113.875000 1 144.000000 2 152...,0 132.5 1 134.0 2 143.5 3 ...,0 119.0 1 118.5 2 118.0 3 ...,0 125.5 1 121.5 2 129.0 3 ...,0 137.0 1 136.5 2 138.5 3 ...,0 55.5 1 52.0 2 77.5 3 ...,0 116.5 1 116.5 2 117.0 3 ...,0 114.0 1 110.5 2 123.5 3 ...,4.0
